In [1]:
!pip install openai gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.9 MB

In [2]:
import openai
import gradio as gr

# Initialize OpenAI API (make sure you have the necessary credentials)
openai.api_key = 'sk-'

step = 0
questions = []
answers = []

def critique_prompt(prompt_text):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f"As an expert prompt engineer, ask questions on the user requirements and provide suggestions for improving the prompt: '{prompt_text}'.",
        max_tokens=500
    )
    # Extract the series of questions (limited to 4 questions max).
    qs = response.choices[0].text.strip().split('\n')[:4]
    return qs


In [3]:
def generate_final_prompt():
    combined_text = ' '.join(answers)
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f"Do not respond to user question/requirement. An important step is to describe the step-by-step process on how the final prompt has been designed, mention the best practices followed in this prompt generation. The final prompt should be effective way to mention user's requirements to chatGPT. Generate the refined prompt, display the final refined prompt based on: '{combined_text}'.",
        max_tokens=1000
    )
    return response.choices[0].text.strip()

In [4]:
def app_interface(prompt_text):
    global step, questions, answers

    # Initial step: get the questions from the model.
    if step == 0:
        questions = critique_prompt(prompt_text)
        step += 1
        return questions[step-1]

    # Intermediate steps: collect user answers and ask subsequent questions.
    elif 0 < step < len(questions):
        answers.append(prompt_text)
        question_to_ask = questions[step]
        step += 1
        return question_to_ask

    # Final step: check if user is ready to generate the final prompt.
    elif step == len(questions):
        answers.append(prompt_text)
        step += 1
        return "Have you given all the necessary details, are we ready to generate the prompt?"

    # Generate refined prompt with explanations.
    else:
        if "yes" in prompt_text.lower():
            final_output = generate_final_prompt()
            # Reset step for future interactions.
            step = 0
            return final_output
        else:
            step = 0
            return "Please provide more details."

In [5]:
interface = gr.Interface(
    fn=app_interface,
    inputs=gr.inputs.Textbox(label="Enter Your Prompt or Answer to the Question"),
    outputs=gr.outputs.Textbox(label="Refined Prompt or Explanation"),
    live=False
)

interface.launch()

<ipython-input-5-4520fa9a8ff2>:3: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Textbox(label="Enter Your Prompt or Answer to the Question"),
<ipython-input-5-4520fa9a8ff2>:3: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(label="Enter Your Prompt or Answer to the Question"),
<ipython-input-5-4520fa9a8ff2>:3: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(label="Enter Your Prompt or Answer to the Question"),
<ipython-input-5-4520fa9a8ff2>:4: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=gr.outputs.Textbox(label="Refined Prompt or Explanation"),


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d7f66d6d7b3ad34419.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [7]:
import gradio as gr

demo = gr.Interface(fn=app_interface, inputs="text", outputs="text")

if __name__ == "__main__":

    demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://460cccac62ae35a51f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
